# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/mtp-thesis

/content/drive/MyDrive/mtp-thesis


# Loading Data

In [4]:
DATA_FILE = '/content/drive/MyDrive/mtp-thesis/data/X_train_YG7NZSq.csv'

In [5]:
RESULTS_FOLDER = '/content/drive/MyDrive/mtp-thesis/results'

In [6]:
import pandas as pd
import numpy as np

In [7]:
_data = pd.read_csv(DATA_FILE)
_datam = np.matrix(_data)

In [8]:
_data.T.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,...,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000,755.000000
mean,0.000156,0.001655,0.002302,0.003973,0.005404,0.006715,0.008140,0.009503,0.010514,0.012255,...,0.054295,0.054405,0.055731,0.056169,0.058028,0.058148,0.060357,0.063013,0.063727,0.064825
std,0.011619,0.037021,0.073401,0.109735,0.145991,0.182294,0.218978,0.254849,0.291350,0.327692,...,1.455853,1.492212,1.528551,1.565071,1.601351,1.637953,1.674178,1.710509,1.746941,1.783311
min,-0.042102,-0.033115,-0.033386,-0.045707,-0.046940,-0.061555,-0.065764,-0.032658,-0.042606,-0.033406,...,-0.082784,-0.056613,-0.024776,-0.094137,-0.044128,-0.228470,-0.058375,-0.053760,-0.050026,-0.027561
25%,-0.006971,-0.003719,-0.005913,-0.006593,-0.005764,-0.006440,-0.009267,-0.004088,-0.005906,-0.005975,...,-0.009494,-0.008544,-0.004325,-0.010665,-0.004565,-0.010652,-0.007956,-0.005281,-0.007516,-0.004662
50%,-0.000168,0.000218,-0.000134,0.000491,0.000524,0.000025,-0.000493,0.000064,-0.000574,0.000281,...,0.000815,-0.000283,0.000480,-0.001489,-0.000403,-0.000872,-0.000194,0.000898,-0.000171,-0.000072
75%,0.007290,0.004654,0.005255,0.006648,0.006462,0.006511,0.008803,0.004773,0.005157,0.006034,...,0.012306,0.009035,0.004388,0.008589,0.004114,0.009286,0.006722,0.006519,0.007489,0.004659
max,0.059061,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,40.000000,41.000000,42.000000,43.000000,44.000000,45.000000,46.000000,47.000000,48.000000,49.000000


In [9]:
_datam.shape

(50, 755)

# Data Handler Class and Utils Function

In [10]:
import numpy as np

def gram_schmidt_algorithm(A):
    """
    O(d^2k)
    """
    for i in range(A.shape[0]):
        q = A[i, :]
        for j in range(i):
            q = q - np.dot(A[j,:], A[i,:]) * A[j,:]
        q = q / np.sqrt(np.dot(q, q))
        A[i,:] = q
    return A

def cayley_transformation(A):
    """
    A is (d,k) matrix
    T = 
    O()
    """
    I = np.eye(A.shape[0])
    Q = np.matmul(np.linalg.inv(I+A),(I-A))
    return Q

def random_matrix(shape, limits_gap=200, center=0.5):
    rmat = np.random.random(shape)
    while np.linalg.matrix_rank(rmat) < min(shape[0],shape[1]):
        rmat = np.random.random(shape)
    return limits_gap*(rmat-center)

class Data_Handler:
    def __init__(self, n, k, d, T, S, R):
        """
        R is a n x (T + S) numpy array
        d is an integer that represents the number of time lags
        d <= k
        """
        self.n = n
        self.k = k
        self.d = d
        self.T = T
        self.S = S
        self.R = R
        assert(d<=k)
        assert(R.shape[0]==n)
        assert(R.shape[1]==(T+S))
        self._compute_optimizers()
        pass

    def get_string(self):
        return f"{self.n}-{self.k}-{self.d}-{self.T}-{self.S}"

    def _compute_optimizers(self):
        self.sum_RtdTRtd_dT_altD = np.zeros((self.d, self.d))  # d x d
        self.sum_RtdTRt_dT_altN = np.zeros((self.d, 1))  # d x 1
        self.list_RtdTRtd = []
        self.list_RtdTRt = []
        self.list_norm_Rt = []
        for t in range(self.d):
            r_t = self.R[:, t].reshape(self.n, 1)  # n x 1
            self.list_RtdTRtd += [None]
            self.list_RtdTRt += [None]
            self.list_norm_Rt += [np.linalg.norm(r_t, 2)]
        for t in range(self.d, self.T+self.S):
            r_t = self.R[:, t].reshape(self.n, 1)  # n x 1
            r_td = self.R[:, t-self.d:t]  # n x d
            self.list_RtdTRtd += [np.matmul(r_td.T, r_td)]
            self.list_RtdTRt += [np.matmul(r_td.T, r_t)]
            self.list_norm_Rt += [np.linalg.norm(r_t, 2)]
        for t in range(self.d, self.T):
            self.sum_RtdTRtd_dT_altD += self.list_RtdTRtd[t]
            self.sum_RtdTRt_dT_altN += self.list_RtdTRt[t]

    def compute_reward(self, beta, A):
        Abeta = np.matmul(A, beta)  # d x 1
        reward_sum = 0
        for t in range(self.T, self.T + self.S):
            reward_sum += np.matmul(self.list_RtdTRt[t].T, Abeta) / (self.list_norm_Rt[t] * np.sqrt(np.matmul(Abeta.T, np.matmul(self.list_RtdTRtd[t], Abeta))))
        reward = reward_sum / self.S
        return reward

    def compute_loss(self, beta, A):
        Abeta = np.matmul(A, beta)  # d x 1
        loss_sum = 0
        for t in range(self.d, self.T):
            loss_sum += self.list_norm_Rt[t]
            loss_sum += -2*np.matmul(self.list_RtdTRt[t].T, Abeta)
            loss_sum += np.matmul(Abeta.T, np.matmul(self.list_RtdTRtd[t], Abeta))
        loss = loss_sum / (2*(self.T - self.d + 1))
        return loss

    def compute_loss_gradient_beta(self, beta, A):
        beta_grad = np.matmul(A.T, np.matmul(self.sum_RtdTRtd_dT_altD, np.matmul(A, beta))) - np.matmul(A.T, self.sum_RtdTRt_dT_altN)
        beta_grad = beta_grad / (self.T - self.d + 1)
        return beta_grad

    def compute_loss_gradient_A(self, beta, A):
        A_grad = np.matmul(self.sum_RtdTRtd_dT_altD, np.matmul(A, beta)) - self.sum_RtdTRt_dT_altN
        A_grad = np.matmul(A_grad, beta.T)
        A_grad = A_grad / (self.T - self.d + 1)
        return A_grad

    def compute_loss_gradient_beta_A(self, beta, A):
        temp = np.matmul(self.sum_RtdTRtd_dT_altD, np.matmul(A, beta))
        beta_grad = np.matmul(A.T, temp) - np.matmul(A.T, self.sum_RtdTRt_dT_altN)
        beta_grad = beta_grad / (self.T - self.d + 1)
        A_grad = temp - self.sum_RtdTRt_dT_altN
        A_grad = np.matmul(A_grad, beta.T)
        A_grad = A_grad / (self.T - self.d + 1)
        return beta_grad, A_grad

    def compute_reward_gradient_beta(self, beta, A):
        T1 = np.zeros((self.d,1))
        T2 = np.zeros((self.d,self.d))
        Abeta = np.matmul(A, beta)  # d x 1
        for t in range(self.T, self.T + self.S):
            Ftbeta_norm = float(np.sqrt(np.matmul(Abeta.T,np.matmul(self.list_RtdTRtd[t],Abeta))))
            RtFtbeta_scalar = float(np.matmul(self.list_RtdTRt[t].T, Abeta))
            T1 += self.list_RtdTRt[t]/(Ftbeta_norm * self.list_norm_Rt[t])
            T2 += (RtFtbeta_scalar/(self.list_norm_Rt[t]*Ftbeta_norm**3)  * self.list_RtdTRtd[t])
        grad_beta = np.matmul(A.T, T1) - np.matmul(A.T, np.matmul(T2,Abeta))
        grad_beta =  grad_beta / self.S
        return grad_beta

    def compute_reward_gradient_A(self, beta, A):
        T1 = np.zeros((self.d,1))
        T2 = np.zeros((self.d,self.d))
        Abeta = np.matmul(A, beta)  # d x 1
        for t in range(self.T, self.T + self.S):
            Ftbeta_norm = float(np.sqrt(np.matmul(Abeta.T,np.matmul(self.list_RtdTRtd[t],Abeta))))
            RtFtbeta_scalar = float(np.matmul(self.list_RtdTRt[t].T, Abeta))
            T1 += self.list_RtdTRt[t]/(Ftbeta_norm * self.list_norm_Rt[t])
            T2 += (RtFtbeta_scalar/(self.list_norm_Rt[t]*Ftbeta_norm**3)  * self.list_RtdTRtd[t])
        grad_A = np.matmul(T1, beta.T) - np.matmul(np.matmul(T2, Abeta), beta.T)
        grad_A =  grad_A / self.S
        return grad_A
    
    def compute_reward_gradient_beta_A(self, beta, A):
        T1 = np.zeros((self.d,1))
        T2 = np.zeros((self.d,self.d))
        Abeta = np.matmul(A, beta)  # d x 1
        for t in range(self.T, self.T + self.S):
            Ftbeta_norm = float(np.sqrt(np.matmul(Abeta.T,np.matmul(self.list_RtdTRtd[t],Abeta))))
            RtFtbeta_scalar = float(np.matmul(self.list_RtdTRt[t].T, Abeta))
            T1 += self.list_RtdTRt[t]/(Ftbeta_norm * self.list_norm_Rt[t])
            T2 += (RtFtbeta_scalar/(self.list_norm_Rt[t]*Ftbeta_norm**3)  * self.list_RtdTRtd[t])
        grad_beta = np.matmul(A.T, T1) - np.matmul(A.T, np.matmul(T2,Abeta))
        grad_beta =  grad_beta / self.S
        grad_A = np.matmul(T1, beta.T) - np.matmul(np.matmul(T2, Abeta), beta.T)
        grad_A =  grad_A / self.S
        return grad_beta, grad_A
    
    def compute_orthogonal_condition_normF(self, A):
        return np.sum((np.matmul(A, A.T) - np.eye(A.shape[0]))**2)

    def compute_orthogonal_condition_normF_gradient_A(self, A):
        return -4.0*np.matmul(np.eye(A.shape[0]) - np.matmul(A, A.T), A)

    def linear_l2_regression(self, A):
        """
        O(d^2k+k^3) = O(k^3)
        """
        beta = np.matmul(np.matmul(A.T,self.sum_RtdTRtd_dT_altD),A)
        beta = np.linalg.inv(beta)
        beta = np.matmul(beta, np.matmul(A.T,self.sum_RtdTRt_dT_altN))
        return beta

# Logger 

In [ ]:
!pip install gif

In [13]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import gif

class Logger:
    def __init__(self, log_name: str, data_handler:Data_Handler):
        self.log_name = log_name
        self.list_R = []
        self.list_L = []
        self.list_O = []
        self.EA_RLO_logs = dict()
        self.EA_archive_RLO = dict()
        self.data_handler = data_handler
        pass

    def clear(self):
        self.list_R = []
        self.list_L = []
        self.list_O = []
        self.EA_RLO_logs = dict()
        self.EA_archive_RLO = dict()
        pass

    def log_RLO(self, R, L, O):
        self.list_L += [float(L)]
        self.list_R += [float(R)]
        self.list_O += [float(O)]
        pass
    
    def log_EA_RLO(self,gen,R,L,O):
        R = float(R)
        L = float(L)
        O = float(O)
        if gen in self.EA_RLO_logs.keys():
            self.EA_RLO_logs[gen][0].append(R)
            self.EA_RLO_logs[gen][1].append(L)
            self.EA_RLO_logs[gen][2].append(O)
        else:
            self.EA_RLO_logs[gen] =  [[R],[L],[O]]
        pass  
        
    def log_EA_Population(self, gen, P): 
        R = [float(P[i].R) for i in range(len(P))]
        L = [float(P[i].L) for i in range(len(P))]
        O = [float(self.data_handler.compute_orthogonal_condition_normF(P[i].A)) for i in range(len(P))]
        self.EA_RLO_logs[gen] = [R,L,O]

    def log_archive(self,gen,Arch):
        R = [float(Arch[i].R) for i in range(len(Arch))]
        L = [float(Arch[i].L) for i in range(len(Arch))]
        O = [float(self.data_handler.compute_orthogonal_condition_normF(Arch[i].A)) for i in range(len(Arch))]
        self.EA_archive_RLO[gen] = [R,L,O]
        pass
    
    def dump(self,folder):
        if len(self.list_L) > 0:
            df = pd.DataFrame()
            df['R'] = self.list_R
            df['L'] = self.list_L
            df['O'] = self.list_O
            fname = "RLO_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            df.to_csv(fpath, index = False)
            print(f" {self.log_name} : dumped ROL")
        
        if len(self.EA_RLO_logs) > 0:
            df = pd.DataFrame()
            for k,v in self.EA_RLO_logs.items():
                df[f"{k}_R"] = v[0]
                df[f"{k}_L"] = v[1]
                df[f"{k}_O"] = v[2]
            fname = "EARLO_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            df.to_csv(fpath, index = False)
            print(f" {self.log_name} : dumped EARLO")

        if len(self.EA_archive_RLO) > 0:
            l = []
            for k,v in self.EA_archive_RLO.items():
                l.append((f"{k}_R", pd.Series(v[0])))
                l.append((f"{k}_L", pd.Series(v[1])))
                l.append((f"{k}_O", pd.Series(v[2])))
            df = pd.DataFrame(dict(l))
            fname = "EAarchiveRLO_"+self.log_name+".csv"
            fpath = os.path.join(folder,fname)
            df.to_csv(fpath, index = False)
            print(f" {self.log_name} : dumped EAarchiveROL")
        
        pass

    def load(self,rlo_file=None, earlo_file=None, eaarchiverl_file=None):
        if rlo_file is not None:
            df = pd.read_csv(rlo_file)
            self.list_R = list(df['R'])
            self.list_L = list(df['L'])
            self.list_O = list(df['O'])
        else:
            self.list_R = []
            self.list_L = []
            self.list_O = []
        
        self.EA_RLO_logs = dict()
        if earlo_file is not None:
            df = pd.read_csv(earlo_file)
            for k in df.columns:
                g = int(k.split('_')[0])
                if g in self.EA_RLO_logs.keys():
                    continue
                self.EA_RLO_logs[g] = [list(df[g+"_R"]),list(df[g+"_L"]),list(df[g+"_O"])]
        
        self.EA_archive_RLO = dict()
        if eaarchiverl_file is not None:
            df = pd.read_csv(eaarchiverl_file)
            for k in df.columns:
                g = int(k.split('_')[0])
                if g in self.EA_archive_RLO.keys():
                    continue
                self.EA_archive_RLO[g] = [list(df[g+"_R"]),list(df[g+"_L"]),list(df[g+"_O"])]
        pass


    def plot_scatter_LR(self, x_scale="linear", y_scale="linear", save=False, file_name="fig.png"):
        sns.set_style("darkgrid")
        sns.set_palette("bright")
        x_label = "Reward"
        y_label = "Loss"
        ax = sns.scatterplot(x=self.list_R, y=self.list_L)
        ax.set(xscale=x_scale, yscale=y_scale, xlabel=x_label, ylabel=y_label)
        if save:
            plt.savefig(file_name)
        plt.show()

    def dist_plot(self, save=False, filename='distplot.png'):
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
        sns.set_style('whitegrid')
        sns.distplot(self.list_R, label='Reward', ax=ax1)
        ax1.set(title='Distribution of Rewards')
        sns.distplot(self.list_L, label='Loss', ax=ax2)
        ax2.set(title='Distribution of Loss')
        ax1.legend()
        ax2.legend()
        if save:
            plt.savefig(filename)
        plt.show()
    
    def get_gif_from_RL_log_dict(self, duration=200,style='seaborn',xscale='log',yscale='log',dpi=300,lam_margins=1.2, set_boundaries = False):
        plt.style.use(style)
        gif.options.matplotlib["dpi"] = dpi

        def get_min_max_limits(lam):
            R_max = -float('inf')
            R_min = float('inf')
            L_max = -float('inf')
            L_min = float('inf')
            for k,v in self.EA_RLO_logs.items():
                R_max = max(np.max(v[0]),R_max)
                R_min = min(np.min(v[0]),R_min)
                L_max = max(np.max(v[1]),L_max)
                L_min = min(np.min(v[1]),L_min)
            return 0.5*np.asarray([  (1+lam)*R_min+(1-lam)*R_max, (1-lam)*R_min+(1+lam)*R_max, (1+lam)*L_min+(1-lam)*L_max, (1-lam)*L_min+(1+lam)*L_max])
            
        limits = get_min_max_limits(lam_margins)
        @gif.frame
        def plot(key,value):
            sns.set(style="whitegrid")
            ax = sns.scatterplot(x=value[0], y=value[1], color='red',)
            ax.set_title(f"{self.log_name} Generation : {key}")
            if set_boundaries:
                ax.set_xlim([limits[0], limits[1]])
                ax.set_ylim([limits[2], limits[3]])

        frames = [ plot(k,v) for k,v in self.EA_RLO_logs.items()]
        filename = f"{self.log_name}" + ".gif"
        gif.save(frames, filename, duration=duration)
        pass

    def get_grid_plot_from_RL_log_dict(self,period,plot_grid_shape):
        pass

    def get_tragectory_gif(self,duration=200,style='seaborn',xscale='log',yscale='log',dpi=300,lam_margins=1.2,set_boundaries = False):
        plt.style.use(style)
        gif.options.matplotlib["dpi"] = dpi

        def get_min_max_limits(lam):
            R_max = -float('inf')
            R_min = float('inf')
            L_max = -float('inf')
            L_min = float('inf')
            for k,v in self.EA_RLO_logs.items():
                R_max = max(np.max(v[0]),R_max)
                R_min = min(np.min(v[0]),R_min)
                L_max = max(np.max(v[1]),L_max)
                L_min = min(np.min(v[1]),L_min)
            return 0.5*np.asarray([  (1+lam)*R_min+(1-lam)*R_max, (1-lam)*R_min+(1+lam)*R_max, (1+lam)*L_min+(1-lam)*L_max, (1-lam)*L_min+(1+lam)*L_max])
        
        limits = get_min_max_limits(lam_margins)
        @gif.frame
        def plot(gen_i):
            fig, ax = plt.subplots()
            for point in range(len(self.EA_RLO_logs[gen_i][0])):
                point_Rs = [self.EA_RLO_logs[gg][0][point] for gg in range(gen_i+1)]
                points_Ls = [self.EA_RLO_logs[gg][1][point] for gg in range(gen_i+1)]
                # ax.plot(point_Rs, points_Ls, label=f'Point {point+1}')
                ax.plot(point_Rs, points_Ls)
                if set_boundaries:
                    ax.set_xlim([limits[0], limits[1]])
                    ax.set_ylim([limits[2], limits[3]])

            # Set plot properties
            ax.set_xlabel('Reward')
            ax.set_ylabel('Loss')
            ax.set_title(f"Trajectories of Points: {gen_i}")
            ax.set_yscale(yscale)
            ax.set_xscale(xscale)

        gen_max = len(self.EA_RLO_logs)
        frames = [ plot(i) for i in range(gen_max)]
        filename = f"{self.log_name}-tragectory" + ".gif"
        gif.save(frames, filename, duration=duration)
        pass

    def show_tragetctory_plot(self,style='seaborn',xscale='log',yscale='log'):
        plt.style.use(style)
        fig, ax = plt.subplots()
        gen_max = len(self.EA_RLO_logs)
        # Plot each point's trajectory as a line
        for point in range(len(self.EA_RLO_logs[0][0])):
            point_Rs = [self.EA_RLO_logs[gg][0][point] for gg in range(gen_max)]
            points_Ls = [self.EA_RLO_logs[gg][1][point] for gg in range(gen_max)]
            ax.plot(point_Rs, points_Ls, label=f'Point {point+1}')

        # Set plot properties
        ax.set_xlabel('Reward')
        ax.set_ylabel('Loss')
        ax.set_title('Trajectories of Points')
        ax.set_yscale(yscale)
        ax.set_xscale(xscale)
        # ax.legend()
        plt.show()

# Baseline Method

## Code

In [14]:
import numpy as np
from tqdm import tqdm

class Baseline_Method:
    def __init__(self, data_handler: Data_Handler):
        self.n = data_handler.n
        self.k = data_handler.k
        self.d = data_handler.d
        self.T = data_handler.T
        self.S = data_handler.S
        self.data_handler = data_handler
        pass

    def get_string(self):
        return "baseline"
        
    def run(self, iter_max=100, logger:Logger=None):
        beta_opt = None
        A_opt = None
        R_opt = -float('inf')
        L_opt = float('inf')
        for iter in tqdm(range(iter_max), desc='Baseline Method'):
            M = random_matrix((self.d, self.k))
            A = gram_schmidt_algorithm(M)
            beta = self.data_handler.linear_l2_regression(A)
            R = self.data_handler.compute_reward(beta, A)
            L = self.data_handler.compute_loss(beta, A)
            if logger is not None:
                logger.log_RLO(R,L,self.data_handler.compute_orthogonal_condition_normF(A))
            if R > R_opt:
                beta_opt = beta
                A_opt = A
                R_opt = R
                L_opt = L
        return beta_opt, A_opt

## Experiments

In [16]:
n = 50
k = 21
d = 20
T = 600
assert(T<_datam.shape[1])
S = _datam.shape[1]-T

dh = Data_Handler(n,k,d,T,S,_datam)

In [19]:
blm = Baseline_Method(dh)
logger_name = blm.get_string()+'-'+dh.get_string()
logger_bml = Logger(logger_name, dh)

In [20]:
beta, A = blm.run(10000,logger_bml)

Baseline Method: 100%|██████████| 10000/10000 [09:55<00:00, 16.78it/s]


In [21]:
dh.compute_orthogonal_condition_normF(A)

2.216484604989007e-28

In [22]:
logger_bml.dump(RESULTS_FOLDER)

 baseline-50-21-20-600-155 : dumped ROL


In [ ]:
logger_bml.plot_scatter_LR('linear','log')

In [ ]:
logger_bml.dist_plot()

# Gradient Based Methods

## Internal Methods

In [24]:
import numpy as np
from tqdm import tqdm

class U_A_Method:
    def __init__(self):
        pass
class U_A_Method_1(U_A_Method):
    def __init__(self, alpha, data_handler:Data_Handler):
        super().__init__()
        self.data_handler = data_handler
        self.itter = alpha[0]
        self.learning_rate = alpha[1]
        self.weight_R = alpha[2]
        self.weight_L = alpha[3]
        self.weight_O = alpha[4]
        pass
    def get_delta_A(self, beta, A):
        R_grad_A = self.data_handler.compute_reward_gradient_A(beta, A)
        L_grad_A = self.data_handler.compute_loss_gradient_A(beta, A)
        O_grad_A = self.data_handler.compute_orthogonal_condition_normF_gradient_A(A)
        delta = self.weight_R*R_grad_A-self.weight_L*L_grad_A-self.weight_O*O_grad_A
        return self.learning_rate*delta
    def get_string(self):
        return f"U_A[{self.itter},{int(10000*self.learning_rate)},{int(10000*self.weight_R)},{int(10000*self.weight_L)},{int(10000*self.weight_O)}]"


class U_beta_Method:
    def __init__(self):
        pass
class U_beta_Method_1(U_beta_Method):
    def __init__(self, phi, data_handler:Data_Handler):
        super().__init__()
        self.data_handler = data_handler
        self.itter = phi[0]
        self.learning_rate = phi[1]
        self.weight_R = phi[2]
        self.weight_L = phi[3]
        pass
    def get_delta_beta(self, beta, A, A_new):
        R_grad_beta = self.data_handler.compute_reward_gradient_beta(beta, A)
        L_grad_beta = self.data_handler.compute_loss_gradient_beta(beta, A)
        delta = self.weight_R*R_grad_beta-self.weight_L*L_grad_beta
        return self.learning_rate*delta
    def get_string(self):
        return f"U_beta[{self.itter},{int(10000*self.learning_rate)},{int(10000*self.weight_R)},{int(10000*self.weight_L)}]"
class U_beta_Method_2(U_beta_Method):
    def __init__(self, phi, data_handler:Data_Handler):
        super().__init__()
        self.data_handler = data_handler
        self.itter = phi[0]
        self.learning_rate = phi[1]
        self.weight_R = phi[2]
        self.weight_L = phi[3]
        self.weight_reg = phi[4]
        pass
    def get_delta_beta(self, beta, A, A_new):
        R_grad_beta = self.data_handler.compute_reward_gradient_beta(beta, A)
        L_grad_beta = self.data_handler.compute_loss_gradient_beta(beta, A)
        beta_reg = self.data_handler.linear_l2_regression(A_new)

        delta = self.weight_R*R_grad_beta-self.weight_L*L_grad_beta
        delta = (1-self.weight_reg)*self.learning_rate*delta
        delta += self.weight_reg*(beta_reg-2*beta) 
        return delta
    def get_string(self):
        return f"U_beta[{self.itter},{int(10000*self.learning_rate)},{int(10000*self.weight_R)},{int(10000*self.weight_L)},{int(10000*self.weight_reg)}]"

class U_O_Method:
    def __init__(self):
        pass
class U_O_Method_1(U_O_Method):
    def __init__(self, lamb, data_handler:Data_Handler):
        super().__init__()
        self.data_handler = data_handler
        self.iter_max = lamb[0]
        self.learning_rate_A = lamb[1]
        self.learning_rate_beta = lamb[2]
        self.weight_reg = lamb[3]
        pass
    def get_new_beta_A(self, beta, A):
        A_i = random_matrix(A.shape)
        A_i = gram_schmidt_algorithm(A_i)
        beta_i = self.data_handler.linear_l2_regression(A_i)
        A_opt = A_i
        beta_opt = beta_i
        R_opt = self.data_handler.compute_reward(beta_opt,A_opt)
        L_opt = self.data_handler.compute_loss(beta_opt,A_opt)
        for iter in tqdm(range(self.iter_max),desc="U_O_Method_1:get_new_beta_A"):
            deltaA = self.learning_rate_A *(A - A_i)
            T_i =  0.5*(np.matmul(deltaA.T,A_i) - np.matmul(A_i.T,deltaA))
            Q = cayley_transformation(T_i)
            A_i = np.matmul(A_i, Q)
            beta_reg = self.data_handler.linear_l2_regression(A_i)
            beta_grad = self.data_handler.compute_reward_gradient_beta(beta_i, A_i)
            beta_i = beta_i + self.learning_rate_beta*((1-self.weight_reg)*beta_grad+self.weight_reg*(beta_reg-beta_i))
            R_i = self.data_handler.compute_reward(beta_i,A_i)
            L_i = self.data_handler.compute_loss(beta_i,A_i)
            if R_i > R_opt:
                A_opt = A_i
                beta_opt = beta_i
                R_opt = R_i
        return beta_opt, A_opt
    def get_string(self):
        return f"U_O[{self.iter_max},{int(10000*self.learning_rate_A)},{int(10000*self.learning_rate_beta)},{int(10000*self.weight_reg)}]"

## OPI Method

### Code

In [25]:
import numpy as np
from tqdm import tqdm

class OPI:
    def __init__(self,data_handler:Data_Handler, U_beta:U_beta_Method, U_A:U_A_Method):
        self.n = data_handler.n
        self.k = data_handler.k
        self.d = data_handler.d
        self.T = data_handler.T
        self.S = data_handler.S
        self.data_handler = data_handler
        self.U_beta = U_beta
        self.U_A = U_A
        pass

    def run(self, iter_max=100, logger:Logger=None):
        A = random_matrix((self.d, self.k))
        A = gram_schmidt_algorithm(A)
        beta = self.data_handler.linear_l2_regression(A)
        beta_opt = beta
        A_opt = A
        R_opt = self.data_handler.compute_reward(beta,A)
        L_opt = self.data_handler.compute_loss(beta,A)
        if logger is not None:
            logger.log_RLO(R_opt,L_opt,self.data_handler.compute_orthogonal_condition_normF(A_opt))
        for iter in tqdm(range(iter_max), desc='OPI Scheme'):
            deltaA = self.U_A.get_delta_A(beta,A)
            T_i =  0.5*(np.matmul(deltaA.T,A) - np.matmul(A.T,deltaA))
            Q = cayley_transformation(T_i)
            AQ = np.matmul(A,Q)
            beta = beta + self.U_beta.get_delta_beta(beta,A,AQ)
            A = AQ
            R = self.data_handler.compute_reward(beta,A)
            L = self.data_handler.compute_loss(beta,A)
            if R > R_opt:
                A_opt = A
                beta_opt = beta
                R_opt = R
                L_opt = L
            if logger is not None:
                logger.log_RLO(R,L,self.data_handler.compute_orthogonal_condition_normF(A))
        return beta_opt, A_opt

    def get_string(self):
        return f"opi-{self.U_beta.get_string()}-{self.U_A.get_string()}"
        


### Experiments

In [37]:
n = 50
k = 21
d = 20
T = 600
assert(T<_datam.shape[1])
S = _datam.shape[1]-T

dh = Data_Handler(n,k,d,T,S,_datam)

In [38]:
iter_u_a_1 = 100
learning_rate_u_a_1 = 1e-2
weight_R_u_a_1 = 0.3
weight_L_u_a_1 = 0.3
weight_O_u_a_1 = 0.3
alpha_u_a_1 = (iter_u_a_1,learning_rate_u_a_1,weight_R_u_a_1,weight_L_u_a_1,weight_O_u_a_1)
u_a_1 = U_A_Method_1(alpha_u_a_1,dh)

#### Run Experiments on method 1

In [33]:
iter_u_beta_1 = 100
learning_rate_u_beta_1 = 1e-2
weight_R_u_beta_1 = 0.5
weight_L_u_beta_1 = 0.5
phi_1 = (iter_u_beta_1,learning_rate_u_beta_1, weight_R_u_beta_1, weight_L_u_beta_1)
u_beta_1 = U_beta_Method_1(phi_1,dh)

opi_1 = OPI(dh,u_beta_1,u_a_1)
logger_name = opi_1.get_string() +'-'+dh.get_string()
logger_opi_1 = Logger(logger_name,dh)

In [30]:
dh.compute_orthogonal_condition_normF(A)

1.9372352650771337e-27

In [39]:
for i in range(10):
  opi_1 = OPI(dh,u_beta_1,u_a_1)
  logger_name =  opi_1.get_string() +'-['+str(i)+']-'+dh.get_string()
  logger_opi_1 = Logger(logger_name,dh)
  beta, A = opi_1.run(1000,logger_opi_1)
  logger_opi_1.dump(RESULTS_FOLDER)

OPI Scheme: 100%|██████████| 1000/1000 [01:20<00:00, 12.42it/s]


 opi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-[0]-50-21-20-600-155 : dumped ROL


OPI Scheme: 100%|██████████| 1000/1000 [01:15<00:00, 13.29it/s]


 opi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-[1]-50-21-20-600-155 : dumped ROL


OPI Scheme: 100%|██████████| 1000/1000 [01:22<00:00, 12.08it/s]


...

 opi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-[7]-50-21-20-600-155 : dumped ROL


OPI Scheme: 100%|██████████| 1000/1000 [01:15<00:00, 13.30it/s]


 opi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-[8]-50-21-20-600-155 : dumped ROL


OPI Scheme: 100%|██████████| 1000/1000 [01:28<00:00, 11.26it/s]

 opi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-[9]-50-21-20-600-155 : dumped ROL


In [ ]:
logger_opi_1.plot_scatter_LR()

In [ ]:
logger_opi_1.dist_plot()

#### Run Experiments on method 2

In [41]:
iter_u_beta_2 = 100
learning_rate_u_beta_2 = 1e-2
weight_R_u_beta_2 = 0.5
weight_L_u_beta_2 = 0.5
weight_reg_u_beta_2 = 0.5
phi_2 = (iter_u_beta_2,learning_rate_u_beta_2, weight_R_u_beta_2, weight_L_u_beta_2,weight_reg_u_beta_2)
u_beta_2 = U_beta_Method_2(phi_2,dh)

opi_2 = OPI(dh,u_beta_2,u_a_1)
logger_name = opi_2.get_string() +'-'+dh.get_string()
logger_opi_2 = Logger(logger_name,dh)

In [42]:
beta, A = opi_2.run(100,logger_opi_2)

OPI Scheme: 100%|██████████| 100/100 [00:05<00:00, 18.74it/s]


In [43]:
dh.compute_orthogonal_condition_normF(A)

6.845526698138814e-28

In [ ]:
for i in range(10):
  opi_2 = OPI(dh,u_beta_2,u_a_1)
  logger_name = opi_2.get_string()+'-['+str(i)+']-'+dh.get_string()
  logger_opi_2 = Logger(logger_name,dh)
  beta, A = opi_2.run(1000,logger_opi_2)
  logger_opi_2.dump(RESULTS_FOLDER)

In [ ]:
logger_opi_2.plot_scatter_LR()

In [ ]:
logger_opi_2.dist_plot()

## DOI Method

### Code

In [50]:
import numpy as np
from tqdm import tqdm

class DOI:
    def __init__(self, data_handler: Data_Handler, U_beta: U_beta_Method, U_A: U_A_Method, U_O: U_O_Method):
        self.n = data_handler.n
        self.k = data_handler.k
        self.d = data_handler.d
        self.T = data_handler.T
        self.S = data_handler.S
        self.data_handler = data_handler
        self.U_beta = U_beta
        self.U_A = U_A
        self.U_O = U_O
        pass

    def get_string(self):
        return f"doi-{self.U_beta.get_string()}-{self.U_A.get_string()}-{self.U_O.get_string()}"

    def run(self, iter_max=100, logger:Logger=None, random_matrix_scale=2):
        A = random_matrix((self.d, self.k),random_matrix_scale)
        beta = self.data_handler.linear_l2_regression(A)
        for iter in tqdm(range(iter_max), desc='DOI Scheme'):
            A_new = A + self.U_A.get_delta_A(beta, A)
            beta = beta + self.U_beta.get_delta_beta(beta, A, A_new)
            A = A_new
            if logger is not None:
                logger.log_RLO(self.data_handler.compute_reward(beta,A),self.data_handler.compute_loss(beta,A), self.data_handler.compute_orthogonal_condition_normF(A))
        beta, A = self.U_O.get_new_beta_A(beta, A)
        return beta, A


### Expriment

In [51]:
n = 50
k = 21
d = 20
T = 600
assert(T<_datam.shape[1])
S = _datam.shape[1]-T

dh = Data_Handler(n,k,d,T,S,_datam)

In [52]:
iter_u_a_1 = 100
learning_rate_u_a_1 = 1e-2
weight_R_u_a_1 = 0.3
weight_L_u_a_1 = 0.3
weight_O_u_a_1 = 0.3
alpha_u_a_1 = (iter_u_a_1,learning_rate_u_a_1,weight_R_u_a_1,weight_L_u_a_1,weight_O_u_a_1)
u_a_1 = U_A_Method_1(alpha_u_a_1,dh)

iter_u_o_1 = 100
learning_rate_A_u_o_1 = 1e-2
learning_rate_beta_u_o_1 = 1e-2
weight_reg_u_o_1 = 0.5
lamb_u_o_1 = (iter_u_o_1, learning_rate_A_u_o_1, learning_rate_beta_u_o_1, weight_reg_u_o_1)
u_o_1 = U_O_Method_1(lamb_u_o_1,dh)

#### Run Experiments on method 1

In [53]:
iter_u_beta_1 = 100
learning_rate_u_beta_1 = 1e-2
weight_R_u_beta_1 = 0.5
weight_L_u_beta_1 = 0.5
phi_1 = (iter_u_beta_1,learning_rate_u_beta_1, weight_R_u_beta_1, weight_L_u_beta_1)
u_beta_1 = U_beta_Method_1(phi_1,dh)

doi_1 = DOI(dh,u_beta_1,u_a_1,u_o_1)
logger_name = doi_1.get_string() + '-'+dh.get_string()
logger_doi_1 = Logger(logger_name,dh)

In [54]:
beta, A = doi_1.run(100,logger_doi_1)

U_O_Method_1:get_new_beta_A: 100%|██████████| 100/100 [00:04<00:00, 22.46it/s]


In [55]:
dh.compute_orthogonal_condition_normF(A)

1.0455176782715557e-28

In [57]:
for i in range(10):
  doi_1 = DOI(dh,u_beta_1,u_a_1,u_o_1)
  logger_name = doi_1.get_string() +'-['+str(i)+']-'+dh.get_string()
  logger_doi_1 = Logger(logger_name,dh)
  beta, A = doi_1.run(1000,logger_doi_1)
  logger_doi_1.dump(RESULTS_FOLDER)

U_O_Method_1:get_new_beta_A: 100%|██████████| 100/100 [00:05<00:00, 19.00it/s]


 doi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-U_O[100,100,100,5000]-[0]-50-21-20-600-155 : dumped ROL


U_O_Method_1:get_new_beta_A: 100%|██████████| 100/100 [00:04<00:00, 22.62it/s]


 doi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-U_O[100,100,100,5000]-[1]-50-21-20-600-155 : dumped ROL


U_O_Method_1:get_new_beta_A: 100%|██████████| 100/100 [00:05<00:00, 18.52it/s]


...

 doi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-U_O[100,100,100,5000]-[7]-50-21-20-600-155 : dumped ROL


U_O_Method_1:get_new_beta_A: 100%|██████████| 100/100 [00:06<00:00, 16.10it/s]


 doi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-U_O[100,100,100,5000]-[8]-50-21-20-600-155 : dumped ROL


U_O_Method_1:get_new_beta_A: 100%|██████████| 100/100 [00:06<00:00, 14.38it/s]


 doi-U_beta[100,100,5000,5000]-U_A[100,100,3000,3000,3000]-U_O[100,100,100,5000]-[9]-50-21-20-600-155 : dumped ROL


In [ ]:
logger_doi_1.plot_scatter_LR()

#### Run Experiments on method 2


In [59]:
iter_u_beta_2 = 100
learning_rate_u_beta_2 = 1e-2
weight_R_u_beta_2 = 0.5
weight_L_u_beta_2 = 0.5
weight_reg_u_beta_2 = 0.5
phi_2 = (iter_u_beta_2,learning_rate_u_beta_2, weight_R_u_beta_2, weight_L_u_beta_2,weight_reg_u_beta_2)
u_beta_2 = U_beta_Method_2(phi_2,dh)

doi_2 = DOI(dh,u_beta_2,u_a_1,u_o_1)
logger_name = doi_2.get_string() +'-'+dh.get_string()
logger_doi_2 = Logger(logger_name,dh)

In [60]:
beta, A = doi_2.run(100,logger_doi_2)

U_O_Method_1:get_new_beta_A: 100%|██████████| 100/100 [00:04<00:00, 21.85it/s]


In [61]:
dh.compute_orthogonal_condition_normF(A)

3.141694284061615e-28

In [ ]:
for i in range(10):
  doi_2 = DOI(dh,u_beta_2,u_a_1,u_o_1)
  logger_name = doi_2.get_string() +'-['+str(i)+']-'+dh.get_string()
  logger_doi_2 = Logger(logger_name,dh)
  beta, A = doi_2.run(1000,logger_doi_2)
  logger_doi_2.dump(RESULTS_FOLDER)

In [ ]:
logger_doi_2.plot_scatter_LR() 

# NO Preference Method

## Code

In [78]:
import numpy as np
from tqdm import tqdm

class MO_NPM_1:
    def __init__(self, data_handler: Data_Handler, learning_rate_A,learning_rate_beta):
        self.n = data_handler.n
        self.k = data_handler.k
        self.d = data_handler.d
        self.T = data_handler.T
        self.S = data_handler.S
        self.data_handler = data_handler
        self.learning_rate_A = learning_rate_A
        self.learning_rate_beta = learning_rate_beta
        pass

    def run(self, iter_max=100, logger:Logger=None):
        A_i = random_matrix((self.d,self.k))
        A_i = gram_schmidt_algorithm(A_i)
        beta_i = self.data_handler.linear_l2_regression(A_i)
        R_i = self.data_handler.compute_reward(beta_i, A_i)
        L_i = self.data_handler.compute_loss(beta_i, A_i)
        A_opt = A_i
        beta_opt = beta_i
        R_opt = R_i
        L_opt = L_i
        for iter in tqdm(range(iter_max), desc="MO_NPM_2"):
            R_grad_beta, R_grad_A = self.data_handler.compute_reward_gradient_beta_A(beta_i, A_i)
            L_grad_beta, L_grad_A = self.data_handler.compute_loss_gradient_beta_A(beta_i, A_i)
            O_grad_A = self.data_handler.compute_orthogonal_condition_normF_gradient_A(A_i)
            deltaA = float(1-R_i)*R_grad_A - float(L_i)*L_grad_A - 0.5*O_grad_A
            deltabeta = float(1-R_i)*R_grad_beta - float(L_i)*L_grad_beta
            T_i =  0.5*self.learning_rate_A *(np.matmul(deltaA.T,A_i) - np.matmul(A_i.T,deltaA))
            Q = cayley_transformation(T_i)
            A_i = np.matmul(A_i, Q)
            beta_i = beta_i + self.learning_rate_beta*deltabeta
            R_i = self.data_handler.compute_reward(beta_i, A_i)
            L_i = self.data_handler.compute_loss(beta_i, A_i)
            if logger is not None:
                logger.log_RLO(R_i,L_i,self.data_handler.compute_orthogonal_condition_normF(A_i))
            if R_i > R_opt:
                A_opt = A_i
                beta_opt = beta_i
                R_opt = R_i
                L_opt = L_i
        return beta_opt, A_opt

    def get_string(self):
        return f"NPM1[{int(1000*self.learning_rate_A)}-{int(1000*self.learning_rate_beta)}]"

class MO_NPM_2:
    def __init__(self, data_handler: Data_Handler,learning_rate_A, learning_rate_beta):
        self.n = data_handler.n
        self.k = data_handler.k
        self.d = data_handler.d
        self.T = data_handler.T
        self.S = data_handler.S
        self.data_handler = data_handler
        self.learning_rate_A = learning_rate_A
        self.learning_rate_beta = learning_rate_beta
        pass

    def run(self, iter_max=100, logger:Logger=None):
        A_i = random_matrix((self.d,self.k))
        A_i = gram_schmidt_algorithm(A_i)
        beta_i = self.data_handler.linear_l2_regression(A_i)
        R_i = self.data_handler.compute_reward(beta_i, A_i)
        L_i = self.data_handler.compute_loss(beta_i, A_i)
        A_opt = A_i
        beta_opt = beta_i
        R_opt = R_i
        L_opt = L_i
        for iter in tqdm(range(iter_max), desc="MO_NPM_2"):
            R_grad_beta, R_grad_A = self.data_handler.compute_reward_gradient_beta_A(beta_i, A_i)
            L_grad_beta, L_grad_A = self.data_handler.compute_loss_gradient_beta_A(beta_i, A_i)
            O_grad_A = self.data_handler.compute_orthogonal_condition_normF_gradient_A(A_i)
            deltaA = float(1-R_i)*R_grad_A - float(L_i)*L_grad_A - 0.5*O_grad_A
            deltabeta = float(1-R_i)*R_grad_beta - float(L_i)*L_grad_beta
            A_i = A_i + self.learning_rate_A * deltaA
            beta_i = beta_i + self.learning_rate_beta*deltabeta
            R_i = self.data_handler.compute_reward(beta_i, A_i)
            L_i = self.data_handler.compute_loss(beta_i, A_i)
            if logger is not None:
                logger.log_RLO(R_i,L_i,self.data_handler.compute_orthogonal_condition_normF(A_i))
            if R_i > R_opt:
                A_opt = A_i
                beta_opt = beta_i
                R_opt = R_i
                L_opt = L_i
        return beta_opt, A_opt
        
    def get_string(self):
        return f"NPM2[{int(1000*self.learning_rate_A)}-{int(1000*self.learning_rate_beta)}]"

## Experiment Method 1

In [67]:
n = 50
k = 21
d = 20
T = 600
assert(T<_datam.shape[1])
S = _datam.shape[1]-T

dh = Data_Handler(n,k,d,T,S,_datam)

In [68]:
learning_rate_A_monpm_1 = 1e-2
learning_rate_beta_monpm_1 = 1e-2

In [69]:
monpm_1 = MO_NPM_1(dh,learning_rate_A_monpm_1,learning_rate_beta_monpm_1)
logger_name = monpm_1.get_string()+'-'+dh.get_string()
logger_monpm_1 = Logger(logger_name,dh)

In [70]:
beta, A = monpm_1.run(iter_max=100,logger=logger_monpm_1)

MO_NPM_2: 100%|██████████| 100/100 [00:04<00:00, 21.63it/s]


In [ ]:
dh.compute_orthogonal_condition_normF(A)

In [ ]:
for i in range(10):
  monpm_1 = MO_NPM_1(dh,learning_rate_A_monpm_1,learning_rate_beta_monpm_1)
  logger_name = monpm_1.get_string()+'-['+str(i)+']-'+dh.get_string()
  logger_monpm_1 = Logger(logger_name, dh)
  beta, A = monpm_1.run(iter_max=1000,logger=logger_monpm_1)
  logger_monpm_1.dump(RESULTS_FOLDER)

In [ ]:
logger_monpm_1.plot_scatter_LR() 

## Experiment Method 2

In [79]:
n = 50
k = 21
d = 20
T = 600
assert(T<_datam.shape[1])
S = _datam.shape[1]-T

dh = Data_Handler(n,k,d,T,S,_datam)

In [80]:
learning_rate_A_monpm_2 = 1e-2
learning_rate_beta_monpm_2 = 1e-2

In [81]:
monpm_2 = MO_NPM_2(dh,learning_rate_A_monpm_2,learning_rate_beta_monpm_2)
logger_name = monpm_2.get_string()+'-'+dh.get_string()
logger_monpm_2 = Logger(logger_name,dh)

In [83]:
beta, A = monpm_2.run(iter_max=100,logger=logger_monpm_2)

MO_NPM_2: 100%|██████████| 100/100 [00:03<00:00, 27.45it/s]


In [ ]:
dh.compute_orthogonal_condition_normF(A)

In [86]:
for i in range(10):
  monpm_2 = MO_NPM_2(dh,learning_rate_A_monpm_2,learning_rate_beta_monpm_2)
  logger_name = monpm_2.get_string()+'-['+str(i)+']-'+dh.get_string()
  logger_monpm_2 = Logger(logger_name,dh)
  beta, A = monpm_2.run(iter_max=1000,logger=logger_monpm_2)
  logger_monpm_2.dump(RESULTS_FOLDER)

MO_NPM_2: 100%|██████████| 1000/1000 [00:38<00:00, 25.67it/s]


 NPM2[10-10]-[0]-50-21-20-600-155 : dumped ROL


MO_NPM_2: 100%|██████████| 1000/1000 [00:37<00:00, 26.67it/s]


 NPM2[10-10]-[1]-50-21-20-600-155 : dumped ROL


MO_NPM_2: 100%|██████████| 1000/1000 [00:40<00:00, 24.81it/s]


...

 NPM2[10-10]-[7]-50-21-20-600-155 : dumped ROL


MO_NPM_2: 100%|██████████| 1000/1000 [00:38<00:00, 26.00it/s]


 NPM2[10-10]-[8]-50-21-20-600-155 : dumped ROL


MO_NPM_2: 100%|██████████| 1000/1000 [00:38<00:00, 25.67it/s]

 NPM2[10-10]-[9]-50-21-20-600-155 : dumped ROL


In [ ]:
logger_monpm_2.plot_scatter_LR() 